# ATL08 Simplifier

In [2]:
!pip install earthpy

     |████████████████████████████████| 1.4 MB 5.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for earthpy: filename=earthpy-0.9.2-py3-none-any.whl size=1365089 sha256=8c9386b5869653f2618df228bfe554d5300c7536d2ce8ccad2a3349128215938
  Stored in directory: /home/jovyan/.cache/pip/wheels/55/5c/61/98ca676a500734ceda6b410eeab50444e00c5298af736725b7
Successfully built earthpy


In [3]:
import h5py
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
# Use geopandas for vector data and rasterio for raster data
import geopandas as gpd
import rasterio as rio
# Plotting extent is used to plot raster & vector data together
from rasterio.plot import plotting_extent

import earthpy as et
import earthpy.plot as ep

%matplotlib inline

In [5]:
!pwd

/home/jovyan/ICESat-2_Snowdepth


In [114]:
os.chdir(os.path.join(et.io.HOME,
                      'ICESat-2_Snowdepth',
                        'data'))
pass_20200805_data = os.path.join('ICESat2_downloads/processed_ATL08_20200805035817_06230806_003_01.h5')
pass_20200810_data = os.path.join('ICESat2_downloads/processed_ATL08_20200810153407_07070802_003_01.h5')
pass_20200908_data = os.path.join('ICESat2_downloads/processed_ATL08_20200908141010_11490802_003_01.h5')
pass_20201006_data = os.path.join('ICESat2_downloads/processed_ATL08_20201006010159_01810906_003_01.h5')
pass_20201011_data = os.path.join('ICESat2_downloads/processed_ATL08_20201011123751_02650902_003_01.h5')
#pass_20201109_data = os.path.join('processed_ATL08_20201109111355_07070902_003_01.h5')

In [22]:
pass_20200805_data

'/ICESat2_downloads/processed_ATL08_20200805035817_06230806_003_01.h5'

In [39]:
files = [pass_20200805_data, pass_20200810_data, pass_20200908_data, pass_20201006_data, pass_20201011_data]

In [9]:
f = h5py.File(pass_20200805_data, 'r')

In [10]:
list(f.keys())

['METADATA',
 'ancillary_data',
 'gt1l',
 'gt1r',
 'gt2l',
 'gt2r',
 'gt3l',
 'gt3r',
 'orbit_info']

In [13]:
gt1l = f['gt1l']
list(gt1l.keys())

['land_segments', 'signal_photons']

In [14]:
gt1l_landsegments = gt1l['land_segments']

In [15]:
list(gt1l_landsegments.keys())

['canopy',
 'delta_time',
 'dem_h',
 'dem_removal_flag',
 'h_dif_ref',
 'last_seg_extend',
 'latitude',
 'layer_flag',
 'longitude',
 'n_seg_ph',
 'psf_flag',
 'segment_landcover',
 'segment_snowcover',
 'segment_watermask',
 'sigma_across',
 'sigma_along',
 'sigma_atlas_land',
 'sigma_h',
 'sigma_topo',
 'surf_type',
 'terrain',
 'terrain_flg']

In [22]:
gt1l_ls_demh = gt1l_landsegments['dem_h']

In [29]:
gt1l_ls_demh.shape

(511,)

In [16]:
gt1l_signalphotons = gt1l['signal_photons']

In [18]:
list(gt1l_signalphotons.keys())

['delta_time']

In [3]:
os.getcwd()

'/home/jovyan/ICESat-2_Snowdepth/data/ICESat2_downloads'

In [11]:
import pyproj


def transform_coord(proj1, proj2, x, y):
    """Transform coordinates from proj1 to proj2 (EPSG num).

    Example EPSG projections:
        Geodetic (lon/lat): 4326
        WGS 84 / UTM zone 11N: 32611
         """
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+str(proj1))
    proj2 = pyproj.Proj("+init=EPSG:"+str(proj2))
    return pyproj.transform(proj1, proj2, x, y)  # convert

In [1]:
def read_atl08(fname, outdir=os.path.join('ATL08_simplified'), bbox=None):  #'/home/jovyan/ICESat-2_Snowdepth/data/ATL08_simplified'
    """Read one ATL08 file and output 6 reduced files. 
    
    Extract variables of interest and separate the ATL08 file 
    into each beam (ground track) and ascending/descending orbits.
    """

    # Each beam is a group
    group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']

    # Loop through beams
    for k, g in enumerate(group):
    
        #-----------------------------------#
        # 1) Read in data for a single beam #
        #-----------------------------------#
        
        data = {}
    
        try:
            # Load vars into memory (include as many as you want)
            with h5py.File(fname, 'r') as fi:
                
                data['h_te_mean'] = fi[g+'/land_segments/terrain/h_te_mean'][:]
                data['h_te_median'] = fi[g+'/land_segments/terrain/h_te_median'][:]
                data['h_te_min'] = fi[g+'/land_segments/terrain/h_te_min'][:]
                data['h_te_max'] = fi[g+'/land_segments/terrain/h_te_max'][:]
                data['latitude'] = fi[g+'/land_segments/latitude'][:]
                data['longitude'] = fi[g+'/land_segments/longitude'][:]
                data['h_te_mode'] = fi[g+'/land_segments/terrain/h_te_mode'][:]
                data['h_te_skew'] = fi[g+'/land_segments/terrain/h_te_skew'][:]
                data['n_te_photons'] = fi[g+'/land_segments/terrain/n_te_photons'][:]
                data['segment_snowcover'] = fi[g+'/land_segments/segment_snowcover'][:]
                data['terrain_slope'] = fi[g+'/land_segments/terrain/terrain_slope'][:]
                data['h_te_uncertainty'] = fi[g+'/land_segments/terrain/h_te_uncertainty'][:]
                data['dem_h'] = fi[g+'/land_segments/dem_h'][:]                
                
                
        except:
            print('skeeping group:', g)
            print('in file:', fname)
            continue
            
        #---------------------------------------------#
        # 2) Filter data according region and quality #
        #---------------------------------------------#
        
        # Select a region of interest
        if bbox:
            lonmin, lonmax, latmin, latmax = bbox
            bbox_mask = (data['longitude'] >= lonmin) & (data['longitude'] <= lonmax) & \
                        (data['latitude'] >= latmin) & (data['latitude'] <= latmax)
        else:
            bbox_mask = np.ones_like(data['latitude'], dtype=bool)  # get all
            
        # Only keep good data (quality flag + threshold + bbox)
        #original mask. We will keep the flagged data for now. 
        #mask = (data['atl06_quality_summary'] == 0) & (np.abs(data['h_li']) < 10e3) & (bbox_mask == 1)
        mask = (np.abs(data['h_te_median']) < 10e4) & (bbox_mask == 1)
        
        # If no data left, skeep
        if not any(mask): continue
        
        # Update data variables
        for k, v in data.items(): data[k] = v[mask]
            
        #----------------------------------------------------#
        # 3) Reproject coords
        #----------------------------------------------------#
              
        # Geodetic lon/lat -> Polar Stereo x/y
        x, y = transform_coord(4326, 32611, data['longitude'], data['latitude'])
        
        data['x'] = x
        data['y'] = y
              
        #-----------------------#
        # 4) Save selected data to CSV #
        #-----------------------#
        #outdir = Path(outdir)    
        #fname = Path(fname)
        #outdir.mkdir(exist_ok=True)
        
        df = pd.DataFrame.from_dict(data, orient="index")
        df = df.transpose()
        #df.name.replace(df, 'pass_20200712_' + g[1:] )
        #print( )
            # Define output dir and file
        #df.to_csv('pass_20200908' + g[1:] + '.csv')
        #df.to_csv(outdir / fname.name.replace('processed', '')+ g[1:] + fname.name.replace('.h5', '.csv'))  #r'20200908'
        #print(f'{fname[28:45]}{g[1:]}.csv')
        df.to_csv(f'/{outdir}/{fname[28:42]}_{g[1:]}.csv')
    
       #-------------------------#
       # 5) Commented out original code to save data to .h5 files #
       #-------------------------#
        
#         outdir = Path(outdir)    
#         fname = Path(fname)
#         outdir.mkdir(exist_ok=True)
#         outfile = outdir / fname.name.replace('.h5', '_' + g[1:] + '.h5')
        
        # Save variables
        #with h5py.File(outfile, 'w') as fo:
        #    for k, v in data.items(): fo[k] = v
         #   print('out ->', outfile)

NameError: name 'os' is not defined

In [95]:
files

['ICESat2_downloads/processed_ATL08_20200805035817_06230806_003_01.h5',
 'ICESat2_downloads/processed_ATL08_20200810153407_07070802_003_01.h5',
 'ICESat2_downloads/processed_ATL08_20200908141010_11490802_003_01.h5',
 'ICESat2_downloads/processed_ATL08_20201006010159_01810906_003_01.h5',
 'ICESat2_downloads/processed_ATL08_20201011123751_02650902_003_01.h5']

In [122]:
outdir

'home/jovyan/ICESat-2_Snowdepth/data/ATL08_simplified'

In [121]:
outdir = 'home/jovyan/ICESat-2_Snowdepth/data/ATL08_simplified'

In [120]:
!pwd

/home/jovyan/ICESat-2_Snowdepth/data


In [124]:
for x in files:
    read_atl08(x, outdir, bbox=None)
    #print(x)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
<ipython-input-11-1dbf8170d844>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pyproj.transform(proj1, proj2, x, 

ATL08_20200805035gt1l.csv


/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
<ipython-input-11-1dbf8170d844>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pyproj.transform(proj1, proj2, x, y)  # convert
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_strin

ATL08_20200805035gt1r.csv
ATL08_20200805035gt2l.csv


/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
<ipython-input-11-1dbf8170d844>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pyproj.transform(proj1, proj2, x, 

...

/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
<ipython-input-11-1dbf8170d844>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pyproj.transform(proj1, proj2, x, 

ATL08_20201011123gt1l.csv
ATL08_20201011123gt1r.csv
ATL08_20201011123gt2l.csv
ATL08_20201011123gt2r.csv


/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
<ipython-input-11-1dbf8170d844>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pyproj.transform(proj1, proj2, x, 

ATL08_20201011123gt3l.csv
ATL08_20201011123gt3r.csv


<ipython-input-11-1dbf8170d844>:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return pyproj.transform(proj1, proj2, x, y)  # convert
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/srv/conda/envs/notebook/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_strin